## Summarize Individual Census Tracts within Designated Places
This notebook will attempt to clip or intersect census tracts inside the 3,4,5, WFH class census desinated places (CDP). The results of that overlay operation will then be assigned a weighted SVI score based on the amount of area they occupy in an individual CDP. These values will then be averaged resulting the arithmetic mean of the 4 thematic rankings and overall summary ranking for each CDP.

In [8]:
# import all the libraries
import numpy as np
import matplotlib as mp
import fiona
import shapely
from shapely.geometry import Polygon, LineString
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
import matplotlib.pyplot as plt
from rasterio.plot import show
import mapclassify

#change default figure size
plt.rcParams['figure.figsize'] = (12,12)

In [9]:
# read in raw census tract data from .zip and convert to geodataframe
svi = gpd.read_file(r"C:\NewMapsPlus\Map698\us-communities-fire\data\SVI2018_US.zip")

In [ ]:
# have a look at the tract data in map form
ax = svi.plot(figsize=(12,12));
ax.set(xlim=(-140,-60), ylim=(20, 60)) # scale the figure with axes values

In [ ]:
# get some metadata on the tracts
svi.info

In [ ]:
# get crs from data
svi.crs

In [ ]:
# read in cdps shapefile
cdps = gpd.read_file(r'C:\NewMapsPlus\Map698\us-communities-fire\notebooks\data\cdps.shp')

In [ ]:
# look at the cdps
ax = cdps.plot()

In [ ]:
# get cdp crs
cdps.crs

### Project Data Frames to same CRS for Overlay

In [ ]:
# get the svi CRS again
print(svi.crs)

In [ ]:
# et the cdps CRS again
print(cdps.crs)

#### Project both dataframes to equidistant conic projection since we'll be doing some overlays and weighting the results spatially

In [ ]:
# set a variable with parameters for USA Contiguous Equidistant
project_params  = '+proj=eqdc +lat_0=39 +lon_0=-96 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs'

# reproject and assign to new variables
cdps_ec = cdps.to_crs(project_params)
svi_ec = svi.to_crs(project_params)

In [ ]:
# overlay reprojected cdps on reprojected svis
fig, ax = plt.subplots()

svi_ec.plot(ax=ax, zorder=1);
cdps_ec.plot(ax=ax, color='red', zorder=2);

ax.set(title='CDPS and SVI Tracts')
ax.set(xlim=(-2500000, 2500000), ylim=(-2000000, 2000000));

### Intersect Tracts and CDPs
Use geopandas overlay method to intersect polygon dataframes

In [ ]:
# input re-projected dataframes using intersection as method of overlay
cdps_svi_int = gpd.overlay(svi_ec, cdps_ec, how='intersection')

In [ ]:
# inspect the output and show all columns
pd.set_option("display.max_columns", None)

cdps_svi_int.head()

In [ ]:
# get info about output
cdps_svi_int.info

In [ ]:
# plot intersection results in the CDP of El Dorado Hills, CA
eldhills = cdps_svi_int.loc[cdps_svi_int['NAME_x'] == 'El Dorado Hills']

eldhills.plot();

In [ ]:
# keep useful columns and add to new dataframe
cdps_svis = cdps_svi_int.filter(['ST','STATE', 'ST_ABBR','STCNTY','COUNTY', 'FIPS','LOCATION', 'AREA_SQMI', 'RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'RPL_THEMES', 
                                'AFFGEOID_x', 'GEOID', 'NAME_x','NAMELSAD_x', 'STUSPS_x', 'STATE_NAME','Acres_x','majority','geometry' ])

In [ ]:
# inspect the dataframe and show all columns
pd.set_option("display.max_columns", None)

cdps_svis.head()

In [ ]:
# write the new dataframe to a shapefile
cdps_svis.to_file(r'C:\NewMapsPlus\Map698\us-communities-fire\data\cdps_svis.shp')